In [ ]:
import pandas as pd
import numpy as np
import os

# Video Statistics Preprocessing

In [ ]:
#Load data
clickBait = pd.read_csv('/content/drive/MyDrive/DataCollection/clickbait.csv', error_bad_lines=False)
notClickBait = pd.read_csv('/content/drive/MyDrive/DataCollection/notClickbait.csv', error_bad_lines=False)

clickBaitImages = []
nonClickBaitImages = []
for image in os.listdir("/content/drive/MyDrive/DataCollection/isClickbait"):
  clickBaitImages.append(image)

for image in os.listdir("/content/drive/MyDrive/DataCollection/notClickbait"):
  nonClickBaitImages.append(image)

In [ ]:
#Initial Cleanup
for like in clickBait["Likes"]:
  clickBait["Dislike-to-like-Ratio"] = clickBait["Dislikes"] / like

for like in notClickBait["Likes"]:
  notClickBait["Dislike-to-like-Ratio"] = notClickBait["Dislikes"] / like

del notClickBait["Favorites"]
del notClickBait["ID"]
del clickBait["Favorites"]
del clickBait["ID"]

clickBait["isClickBait"] = 1
notClickBait["isClickBait"] = 0

In [ ]:
# Combining the data
data = pd.concat([clickBait, notClickBait])
data.sample(frac=1, random_state=42) #shuffling

,Video Title,Views,Likes,Dislikes,Dislike-to-like-Ratio,isClickBait
95,SHE IS PREGNANT!!,6021486,217166,4904,0.002575,1
15,BRENT AND I PRANK OUR MOM!!,15114182,375282,7265,0.003814,1
30,kid shoots dad after he takes PS5..,29079,2177,368,0.000193,1
57,DaBaby - Wockesha (Freestyle) [Official Video],1823136,137724,3211,0.233970,0
27,Qualifying Highlights 2021 Dutch Grand Prix,3304392,81058,930,0.067765,0
...,...,...,...,...,...,...
5,Turning cotton balls into cotton candy,1609573,131024,704,0.051297,0
14,"Cutting Peoples Earphones, Then Giving Them Ai...",43680379,1297208,40894,0.021471,1
92,DONT PLAY GTA 5 AT 3:00 AM.. (scary),2974379,78932,10362,0.005440,1
78,OFFICIAL SCIENTIST Mod in Among Us,3607286,238848,3356,0.244535,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, data["isClickBait"], test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler

stats = X_train[["Views", "Likes", "Dislikes"]]
scaler = StandardScaler()
scaler.fit(stats)
stats_scaled = pd.DataFrame(scaler.transform(stats))
X_train[["Views", "Likes", "Dislikes"]] = stats_scaled

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
stats2 = X_test[["Views", "Likes", "Dislikes"]]
scaler2 = StandardScaler()
scaler2.fit(stats2)
stats_scaled = pd.DataFrame(scaler2.transform(stats2))
X_test[["Views", "Likes", "Dislikes"]] = stats_scaled
X_test.fillna(0,inplace=True)
X_test.head(10)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,Video Title,Views,Likes,Dislikes,Dislike-to-like-Ratio,isClickBait
48,Inside Japan's WORST Hotel Room Feat. @CDawgVA,0.000000,0.000000,0.000000,0.048528,0
57,SHARK ATTACK PRANK (WIFE FREAKOUT),0.000000,0.000000,0.000000,0.019575,1
24,my girlfriend forgot to end her stream..,-0.584913,-0.336538,-0.311758,0.000138,1
31,i shouldn't have let him do this..,1.559676,0.014978,-0.200410,0.000035,1
81,10 Actors Who Turned Into Monsters,0.000000,0.000000,0.000000,0.027450,1
18,this video will make you forget your name..,-0.370385,-0.363224,-0.308975,0.001829,1
27,he fell off then died..,2.812748,-0.095973,0.341590,0.000287,1
58,I Flew To MrBeast's House & Knocked On His Door,0.000000,0.000000,0.000000,0.077628,1
8,I Went Over Speed House And My Brothers Pulled...,-0.604287,-0.329286,-0.312727,0.000050,1
50,vegan eats MEAT for 24 hours...,0.000000,0.000000,0.000000,0.046561,0


# ML Model Shortlisting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
knn = KNeighborsClassifier(n_neighbors=5)
svc = SVC()
xgb = xgboost.XGBClassifier(use_label_encoder=False)
log_reg = LogisticRegression()
nb = GaussianNB()


ml_models = [rf,knn,svc, xgb, log_reg, nb]
for ml in ml_models:
    ml.fit(X_train[["Views", "Likes", "Dislikes", "Dislike-to-like-Ratio"]], y_train)
    y_pred = ml.predict(X_test[["Views", "Likes", "Dislikes", "Dislike-to-like-Ratio"]])
    print(accuracy_score(y_pred,y_test))

0.8048780487804879
0.7317073170731707
0.4634146341463415
0.7560975609756098
0.5853658536585366
0.6341463414634146


In [ ]:
from sklearn.ensemble import VotingClassifier

ens = VotingClassifier(estimators=[('rf', rf), ('knn', knn), ('xgb', xgb)], voting="soft")
ens.fit(X_train[["Views", "Likes", "Dislikes", "Dislike-to-like-Ratio"]], y_train)
pred = ens.predict(X_test[["Views", "Likes", "Dislikes", "Dislike-to-like-Ratio"]])
print(accuracy_score(pred, y_test))

0.7804878048780488


In [47]:
import pickle

pickle.dump(ens, open("Stats_Estimator_Ensemble.sav", 'wb'))